In [19]:
# imports to process data using DASK
from dask import delayed
import dask.dataframe as ddf
from dask.diagnostics import ProgressBar

# imports for data analysis and visualization
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# imports for downloading data from FTP site
import os
from ftplib import FTP


# imports to perform spatial aggregation using ArcGIS GeoAnalytics server
#from arcgis.gis import GIS
#from arcgis.geoanalytics import get_datastores
#from arcgis.geoanalytics.summarize_data import reconstruct_tracks
#import arcgis

# miscellaneous imports
from pprint import pprint
from copy import deepcopy

### ADT HURSAT Data Based on Kossin et al (2020) 

In [4]:
!ls 'ADT_HURSAT'/*.csv | head

ADT_HURSAT/CleanDataKossinLongFormat.csv
ADT_HURSAT/DataKossinLongFormat.csv
ADT_HURSAT/DataKossinLongFormatNonNA.csv
ADT_HURSAT/dfKossinNewCat.csv
ADT_HURSAT/pnas.1920849117.sd01.csv
ADT_HURSAT/pnas.1920849117.sd02.csv
ADT_HURSAT/pnas.1920849117.sd03.csv
ADT_HURSAT/pnas.1920849117.sd04.csv
ADT_HURSAT/pnas.1920849117.sd05.csv
ADT_HURSAT/pnas.1920849117.sd06.csv


In [6]:
# try this on csv files exceeding 1M 
dfAdt=ddf.read_csv('ADT_HURSAT/pnas.1920849117.*.csv',include_path_column=False,blocksize='10M') 
dfAdtCompute  = dfAdt.compute()
dfAdtCompute

,Basin,Day1,Day2,Day3,Day4,Day5,Day6,Day7,Day8,Day9,...,Year291,Year292,Year293,Year294,Year295,Year296,Year297,Year298,Year299,Year300
0,EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
dfStormId = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd07.csv')  
dfBasin = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd01.csv') 
dfLat = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd04.csv') 
dfLong = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd05.csv') 
dfWindSpeed = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd08.csv') 
dfYear = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd09.csv') 
dfMonth = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd06.csv') 
dfDay = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd02.csv') 
dfHour = pd.read_csv('ADT_HURSAT/pnas.1920849117.sd03.csv') 

In [8]:
dfStormId.shape

(4180, 1)

In [4]:
frames = [dfStormId, dfBasin, dfLat,dfLong,dfWindSpeed,dfYear,dfMonth,dfDay,dfHour]
result = pd.concat(frames,axis=1)

In [10]:
result

,StormID,Basin,Latitude1,Latitude2,Latitude3,Latitude4,Latitude5,Latitude6,Latitude7,Latitude8,...,Hour291,Hour292,Hour293,Hour294,Hour295,Hour296,Hour297,Hour298,Hour299,Hour300
0,1978151N15260,EP,15.30,15.30,15.30,15.60,16.20,16.50,17.76,18.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1978168N11242,EP,11.20,11.51,12.40,12.62,13.54,14.13,15.00,15.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1978168N14254,EP,13.71,13.71,13.71,13.69,13.41,13.30,13.17,12.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1978173N25274,NaN,25.20,25.20,25.20,25.60,26.00,26.11,26.50,26.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1978178N14260,EP,13.77,13.77,13.77,13.77,14.00,14.33,15.48,15.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4175,2017333N06082,NI,6.00,6.00,6.00,6.00,5.90,6.10,6.20,6.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4176,2017340N09089,NI,8.50,8.80,9.80,10.00,11.10,12.20,12.80,13.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4177,2017347N11129,WP,10.90,10.90,10.90,10.90,10.90,11.25,11.39,11.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4178,2017354N08134,WP,8.20,8.20,8.20,8.20,8.20,8.50,8.69,8.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
DFNonNA = result[result.iloc[:,0:2102].notna()]
#DFNonNA.shape
DFNonNA = result[~result['Latitude1'].isnull()] #just comment this for all data including NaN

#DFNonNA

In [11]:
#pd.wide_to_long(result, stubnames='Latitude', i=['StormID', 'Basin'], j='Lat')

In [5]:
Lat = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,2:302],\
        var_name='LatID',value_name='LAT',ignore_index=False)

Lat = Lat[~Lat['LAT'].isnull()] #just comment this for all data including NaN
#Lat

In [6]:
Long = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,302:602],\
        var_name='LongID',value_name='LONG',ignore_index=False)
Long = Long[~Long['LONG'].isnull()]
#Long

In [7]:
WindSpeed = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,602:902],\
        var_name='SpeedID',value_name='STORM_SPEED',ignore_index=False)

WindSpeed = WindSpeed[~WindSpeed['STORM_SPEED'].isnull()]
#WindSpeed


In [8]:
Year = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,902:1202],\
        var_name='YearID',value_name='YEAR',ignore_index=False)
Year = Year[~Year['YEAR'].isnull()]
#Year

In [9]:
Month = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,1202:1502],\
        var_name='MonthID',value_name='MONTH',ignore_index=False)
Month = Month[~Month['MONTH'].isnull()]
#Month

In [10]:
Day = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,1502:1802],\
        var_name='DayID',value_name='DAY',ignore_index=False)
Day = Day[~Day['DAY'].isnull()]
#Day

In [11]:
Hour = pd.melt(result, id_vars=['StormID','Basin'],value_vars=result.iloc[:,1802:2102],\
        var_name='HourID',value_name='HOUR',ignore_index=False)
Hour = Hour[~Hour['HOUR'].isnull()]
#Hour

In [12]:
#framesWide1 = [[Lat['LAT']], [Long['LONG']],[Year['YEAR']],[Month['MONTH']],[Day['DAY']],[Hour['HOUR']]]
framesWide1 = [(Lat['StormID']),(Lat['Basin']),(Lat['LAT']), (Long['LONG']), (Year['YEAR']), (Month['MONTH']), (Day['DAY']), (Hour['HOUR'])]
resultWide1 = pd.concat(framesWide1,axis=1)
#resultWide1 = pd.concat(framesWide1,join="inner")
resultWide1


,StormID,Basin,LAT,LONG,YEAR,MONTH,DAY,HOUR
0,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0
1,1978168N11242,EP,11.20,-118.00,1978.0,6.0,17.0,0.0
2,1978168N14254,EP,13.71,-107.84,1978.0,6.0,17.0,0.0
3,1978173N25274,NaN,25.20,-86.50,1978.0,6.0,21.0,0.0
4,1978178N14260,EP,13.77,-100.57,1978.0,6.0,26.0,0.0
...,...,...,...,...,...,...,...,...
2149,1998002S14151,SP,-12.20,143.60,1998.0,2.0,2.0,12.0
2037,1997013S08101,SI,-13.90,53.90,1997.0,2.0,14.0,18.0
2149,1998002S14151,SP,-12.20,143.60,1998.0,2.0,2.0,18.0
2037,1997013S08101,SI,-13.90,53.90,1997.0,2.0,15.0,0.0


In [13]:
# ADD STORM CATEGORY
#resultWideMerge["CATEGORY"] = resultWideMerge["STORM_SPEED"] 
#0 = Tropical storm [34<W<64]
#1 = Category 1 [64<=W<83]
#2 = Category 2 [83<=W<96] 
#3 = Category 3 [96<=W<113] 
#4 = Category 4 [113<=W<137] 
#5 = Category 5 [W >= 137] 

bins = [24,34, 64, 83, 96, 113,137]
names = ['0','1', '2', '3', '4', '5']

WindSpeed['CATEGORY'] = WindSpeed['STORM_SPEED'].apply(lambda x: x if isinstance(x, str) else pd.cut([x], bins, labels=names)[0])

WindSpeed


,StormID,Basin,SpeedID,STORM_SPEED,CATEGORY
0,1978151N15260,EP,WindSpeed1,35.0,1
1,1978168N11242,EP,WindSpeed1,25.0,0
2,1978168N14254,EP,WindSpeed1,25.0,0
3,1978173N25274,NaN,WindSpeed1,25.0,0
4,1978178N14260,EP,WindSpeed1,25.0,0
...,...,...,...,...,...
2149,1998002S14151,SP,WindSpeed131,30.0,0
2037,1997013S08101,SI,WindSpeed132,30.0,0
2149,1998002S14151,SP,WindSpeed132,30.0,0
2037,1997013S08101,SI,WindSpeed133,30.0,0


In [14]:
#SAVE TO CSV FOR FURTHER PROCESS USING R
resultWide1.to_csv('ADT_HURSAT/resultWide1.csv')
WindSpeed.to_csv('ADT_HURSAT/WindSpeed.csv')

In [24]:
# HOW TO ADD A COLUMN TO EXISISTING DATASET WITH DIFFERENT NUMBER OF ROWS
#resultWide1['STORM_SPEED'] = WindSpeed['STORM_SPEED']

In [16]:
frame = [(WindSpeed['StormID']),(WindSpeed['STORM_SPEED']),(WindSpeed['CATEGORY'])]
dfWs = pd.concat(frame,axis=1)


In [23]:
resultWideAll = resultWide1.merge(dfWs, on = 'StormID',how='inner')
#df2 = df2.merge(df1, on='m/z',how='left')
#print(df2)

In [24]:
resultWideAll

,StormID,Basin,LAT,LONG,YEAR,MONTH,DAY,HOUR,STORM_SPEED,CATEGORY
0,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
1,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
2,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
3,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
4,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,40.0,1
...,...,...,...,...,...,...,...,...,...,...
5996951,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,25.0,0
5996952,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0
5996953,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0
5996954,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,45.0,1


In [25]:
len(resultWideAll['StormID'].unique())

4158

In [26]:
sum(resultWideAll['STORM_SPEED'].isnull())

0

In [33]:

resultWideMerge = resultWideAll.rename(
    columns={
        "Basin_x": "BASIN",
        "STORMID": "SID"
    }
)
resultWideMerge


,SID,BASIN,LAT,LONG,YEAR,MONTH,DAY,HOUR,STORM_SPEED,CATEGORY
0,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
1,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
2,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
3,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1
4,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,40.0,1
...,...,...,...,...,...,...,...,...,...,...
5996951,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,25.0,0
5996952,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0
5996953,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0
5996954,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,45.0,1


In [34]:
resultWideMerge.to_csv('ADT_HURSAT/ADT_HURSAT_CleanData.csv')

In [59]:
dfK = pd.read_csv('ADT_HURSAT/ADT_HURSAT_CleanData.csv')
dfK


,Unnamed: 0,SID,BASIN,LAT,LONG,YEAR,MONTH,DAY,HOUR,STORM_SPEED,CATEGORY
0,0,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0
1,1,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0
2,2,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0
3,3,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0
4,4,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,40.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
5996951,5996951,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,25.0,0.0
5996952,5996952,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0.0
5996953,5996953,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0.0
5996954,5996954,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,45.0,1.0


In [60]:
#from datetime import datetime
ISO_TIME = pd.to_datetime(dfK[['YEAR', 'MONTH', 'DAY']])
ISO_TIME

pd.to_datetime(dfK[['YEAR', 'MONTH', 'DAY']]) + pd.to_timedelta(dfK.HOUR, unit='h')

0         1978-05-30
1         1978-05-30
2         1978-05-30
3         1978-05-30
4         1978-05-30
             ...    
5996951   2017-12-29
5996952   2017-12-29
5996953   2017-12-29
5996954   2017-12-29
5996955   2017-12-29
Length: 5996956, dtype: datetime64[ns]

In [70]:
dfIso = pd.DataFrame({
    'Day': np.array(ISO_TIME, dtype=np.datetime64), 
    'Hour': np.array(dfK.HOUR, dtype=np.int64)})

ISO_TIME = pd.to_datetime(dfIso.Day) + pd.to_timedelta(dfIso.Hour, unit='h')
ISO_TIME

0         1978-05-30
1         1978-05-30
2         1978-05-30
3         1978-05-30
4         1978-05-30
             ...    
5996951   2017-12-29
5996952   2017-12-29
5996953   2017-12-29
5996954   2017-12-29
5996955   2017-12-29
Length: 5996956, dtype: datetime64[ns]

In [87]:
dfK['ISO_TIME'] = ISO_TIME
dfK

,SID,BASIN,LAT,LONG,YEAR,MONTH,DAY,HOUR,STORM_SPEED,CATEGORY,ISO_TIME
0,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
1,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
2,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
3,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
4,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,40.0,1.0,1978-05-30
...,...,...,...,...,...,...,...,...,...,...,...
5996951,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,25.0,0.0,2017-12-29
5996952,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0.0,2017-12-29
5996953,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0.0,2017-12-29
5996954,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,45.0,1.0,2017-12-29


In [80]:
dfK.to_csv('ADT_HURSAT/ADT_HURSAT_CleanData_v02.csv')

In [85]:
import pandas as pd
ADT_HURSAT = pd.read_csv('ADT_HURSAT/ADT_HURSAT_CleanData_v02.csv', names=['SID', 'BASIN', 'YEAR', 'CATEGORY'])
print(ADT_HURSAT.head())

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (3,4,5,6,7,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


                                                               SID  \
NaN       SID           BASIN LAT  LONG    YEAR   MONTH DAY   HOUR   
0.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0   0.0   
1.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0   0.0   
2.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0   0.0   
3.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0   0.0   

                                                                    BASIN  \
NaN       SID           BASIN LAT  LONG    YEAR   MONTH DAY   STORM_SPEED   
0.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0         35.0   
1.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0         35.0   
2.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0         35.0   
3.0       1978151N15260 EP    15.3 -100.25 1978.0 5.0   30.0         35.0   

                                                                  YEAR  \
NaN       SID           BASIN LAT  LONG   

In [91]:
dfK

,SID,BASIN,LAT,LONG,YEAR,MONTH,DAY,HOUR,STORM_SPEED,CATEGORY,ISO_TIME
0,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
1,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
2,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
3,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,35.0,1.0,1978-05-30
4,1978151N15260,EP,15.30,-100.25,1978.0,5.0,30.0,0.0,40.0,1.0,1978-05-30
...,...,...,...,...,...,...,...,...,...,...,...
5996951,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,25.0,0.0,2017-12-29
5996952,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0.0,2017-12-29
5996953,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,30.0,0.0,2017-12-29
5996954,2017360S14124,SI,-21.35,122.87,2017.0,12.0,29.0,0.0,45.0,1.0,2017-12-29
